## Overview

Recall the overall workflow for retrieval augmented generation (RAG):

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

Let's load our vectorDB. 

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
load_dotenv(dotenv_path="/Users/pulkitaggarwal/LangChain2/Gemini_Key.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("Hello Gemini")
print(response.text)

Hello there! How can I help you today?



In [2]:
from langchain.vectorstores import Chroma
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
persist_directory = 'docs/chroma/'
#embedding = OpenAIEmbeddings()
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/wd/hxgvmlms6j39627ykffdygcm0000gn/T/ipykernel_83245/3543508353.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [3]:
print(vectordb._collection.count())

208


In [4]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)


### RetrievalQA chain

In [6]:
from langchain.chains import RetrievalQA

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [8]:
result = qa_chain({"query": question})

/var/folders/wd/hxgvmlms6j39627ykffdygcm0000gn/T/ipykernel_83245/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [9]:
result["result"]

'The class will cover four major topics, the first of which is supervised learning.  The other three topics are not specified in the provided text.'

### Prompt

In [10]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [11]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [12]:
question = "Is probability a class topic?"

In [13]:
result = qa_chain({"query": question})

In [14]:
result["result"]

'Yes, the instructor assumes familiarity with basic probability and statistics.  Review will be provided in discussion sections. Thanks for asking!'

In [15]:
result["source_documents"][0]

Document(metadata={'creationdate': '2008-07-11T11:25:23-07:00', 'page': 4, 'title': '', 'total_pages': 22, 'author': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'moddate': '2008-07-11T11:25:23-07:00', 'page_label': '5', 'creator': 'PScript5.dll Version 5.2.2', 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture01.pdf'}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over 

### RetrievalQA chain types

In [16]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [17]:
result = qa_chain_mr({"query": question})

In [18]:
result["result"]

'The provided text does not contain information about whether probability is a class topic.'

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [19]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [20]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, the instructor assumes familiarity with basic probability and statistics.  A refresher will be provided in discussion sections.'

In [21]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites of basic probability and statistics, and basic linear algebra are needed because the class will use these concepts.  The instructor assumes students know what random variables, expectation, and variance are, as well as matrix and vector operations and inverses.  While a refresher will be provided for some topics, a foundational understanding is expected.'

Note, The LLM response varies. Some responses **do** include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next section.